Config e imports

In [3]:
import requests
import json
from pprint import pprint

BASE_URL = "https://tarea-2-despliegue-modelo-ml-como-api-2.onrender.com"

def pretty(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))


Healthcheck

In [4]:
r = requests.get(f"{BASE_URL}/health", timeout=30)
print("Status:", r.status_code)
pretty(r.json())


Status: 200
{
  "status": "ok",
  "models_dir": "model/artifacts",
  "models": [
    "RF",
    "GB",
    "LR"
  ]
}


## Prueba 1: /predict (xbox, tipo 5)

In [6]:
payload1 = {
    "precio_inicial": 45,
    "n_postores": 5,
    "n_pujas": 10,
    "reputacion_prom": 50,
    "producto": "xbox",
    "tipo_subasta": 5
}

r = requests.post(f"{BASE_URL}/predict", json=payload1, timeout=60)
print("Status:", r.status_code)
print("Payload enviado:")
pretty(payload1)
print("Respuesta:")
pretty(r.json())


Status: 200
Payload enviado:
{
  "precio_inicial": 45,
  "n_postores": 5,
  "n_pujas": 10,
  "reputacion_prom": 50,
  "producto": "xbox",
  "tipo_subasta": 5
}
Respuesta:
{
  "predicciones": {
    "RF": 133.6523,
    "GB": 110.4041,
    "LR": 21.8221
  }
}


## Prueba 2: /predict (palm, tipo 7)

In [7]:
payload2 = {
    "precio_inicial": 120,
    "n_postores": 3,
    "n_pujas": 15,
    "reputacion_prom": 70,
    "producto": "palm",
    "tipo_subasta": 7
}

r = requests.post(f"{BASE_URL}/predict", json=payload2, timeout=60)
print("Status:", r.status_code)
print("Payload enviado:")
pretty(payload2)
print("Respuesta:")
pretty(r.json())


Status: 200
Payload enviado:
{
  "precio_inicial": 120,
  "n_postores": 3,
  "n_pujas": 15,
  "reputacion_prom": 70,
  "producto": "palm",
  "tipo_subasta": 7
}
Respuesta:
{
  "predicciones": {
    "RF": 244.2707,
    "GB": 467.34,
    "LR": 119.6121
  }
}


## Prueba 3: /simulate (100 juegos)

In [8]:
payload3 = {
    "precio_inicial": 100,
    "n_postores": 3,
    "n_pujas": 20,
    "reputacion_prom": 50,
    "producto": "xbox",
    "tipo_subasta": 3,
    "juegos": 100,
    "valor_min": 50,
    "valor_max": 90,
    "incremento": 1,
    "tope": 100,
    "seed": 123
}

r = requests.post(f"{BASE_URL}/simulate", json=payload3, timeout=120)
print("Status:", r.status_code)
print("Payload enviado:")
pretty(payload3)
print("Respuesta (resumen):")
resp = r.json()
pretty({
    "resultados": resp.get("resultados"),
    "ganador_ultimo": resp.get("detalle_ultimo_juego", {}).get("ganador"),
    "precio_pagado_ultimo": resp.get("detalle_ultimo_juego", {}).get("precio_pagado")
})


Status: 200
Payload enviado:
{
  "precio_inicial": 100,
  "n_postores": 3,
  "n_pujas": 20,
  "reputacion_prom": 50,
  "producto": "xbox",
  "tipo_subasta": 3,
  "juegos": 100,
  "valor_min": 50,
  "valor_max": 90,
  "incremento": 1,
  "tope": 100,
  "seed": 123
}
Respuesta (resumen):
{
  "resultados": {
    "RF": 55,
    "GB": 45,
    "LR": 0
  },
  "ganador_ultimo": "GB",
  "precio_pagado_ultimo": 68.0
}


## Manejo básico de errores

In [9]:
bad_payload = {
    "precio_inicial": -10,  # inválido (debería ser > 0)
    "n_postores": 0,        # inválido (>=1)
    "n_pujas": 0,           # inválido (>=1)
    "reputacion_prom": 150, # inválido (0..100)
    "producto": "otro",
    "tipo_subasta": 2
}

r = requests.post(f"{BASE_URL}/predict", json=bad_payload)
print("Status:", r.status_code)
print("Respuesta de validación:")
pretty(r.json())


Status: 422
Respuesta de validación:
{
  "detail": [
    {
      "type": "greater_than",
      "loc": [
        "body",
        "precio_inicial"
      ],
      "msg": "Input should be greater than 0",
      "input": -10,
      "ctx": {
        "gt": 0.0
      }
    },
    {
      "type": "greater_than_equal",
      "loc": [
        "body",
        "n_postores"
      ],
      "msg": "Input should be greater than or equal to 1",
      "input": 0,
      "ctx": {
        "ge": 1
      }
    },
    {
      "type": "greater_than_equal",
      "loc": [
        "body",
        "n_pujas"
      ],
      "msg": "Input should be greater than or equal to 1",
      "input": 0,
      "ctx": {
        "ge": 1
      }
    },
    {
      "type": "less_than_equal",
      "loc": [
        "body",
        "reputacion_prom"
      ],
      "msg": "Input should be less than or equal to 100",
      "input": 150,
      "ctx": {
        "le": 100.0
      }
    },
    {
      "type": "literal_error",
      "loc":